In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchsummary import summary
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, Dataset
from collections import OrderedDict
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#====================================================================
# calling data
trans = transforms.Compose([transforms.Resize((224,224)),
                            transforms.RandomHorizontalFlip(p=0.5), #data augmentation
                            transforms.RandomVerticalFlip(p=0.5),   #data augmentation
                            transforms.RandomAffine(15),            #data augmentation
                            transforms.ToTensor(), 
                            transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))])
trainset = torchvision.datasets.ImageFolder(root='/home/gus/catvsdog',
                                            transform=trans)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=16,
                                          shuffle=True,
                                          drop_last=True,
                                          num_workers=2)
testset = torchvision.datasets.ImageFolder(root='/home/gus/cattest', 
                                           transform=trans)
testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=16,
                                         shuffle=True,
                                         drop_last=True,
                                         num_workers=2)
#====================================================================
# declaring hyperparameters
lr = 1e-3
#====================================================================
# constructiong AlexNet model
class ZFNet(nn.Module):
    def __init__(self):
        super(ZFNet,self).__init__()

        self.features = nn.Sequential(OrderedDict([                    # input shape = (?, 3, 224, 224)
            ('Conv1',nn.Conv2d(3,96,7,stride=2,padding=1)),            # Conv -> (?, 96, 110, 110)
            ('Act1',nn.ReLU()), 
            ('MPool1',nn.MaxPool2d(3,2,padding=1,return_indices=True)),  # Pool -> (?, 96, 55, 55)
            ('LRN1',nn.LocalResponseNorm(256,alpha=1e-4,beta=0.75,k=2)), 
            ('Conv2',nn.Conv2d(96,256,5,stride=2,padding=0)),            # Conv -> (?, 256, 26, 26)
            ('Act2',nn.ReLU()), 
            ('MPool2',nn.MaxPool2d(3,2,padding=1,return_indices=True)),  # Pool -> (?, 256, 13, 13)            
            ('LRN2',nn.LocalResponseNorm(256,alpha=1e-4,beta=0.75,k=2)),
            ('Conv3',nn.Conv2d(256,384,3,stride=1,padding=1)),            # Conv -> (?, 384, 13, 13)
            ('Act3',nn.ReLU()),        
            ('Conv4',nn.Conv2d(384,384,3,stride=1,padding=1)),            # Conv -> (?, 384, 13, 13)
            ('Act4',nn.ReLU()),        
            ('Conv5',nn.Conv2d(384,256,3,stride=1,padding=1)),            # Conv -> (?, 256, 13, 13)
            ('Act5',nn.ReLU()),        
            ('MPool5',nn.MaxPool2d(3,2,padding=0,return_indices=True)),  # Pool -> (?, 256, 6, 6)
        ]))
        
        nn.init.normal_(self.features[0].weight,mean=0,std=0.01)
        nn.init.constant_(self.features[0].bias,0)        
        nn.init.normal_(self.features[4].weight,mean=0,std=0.01)
        nn.init.constant_(self.features[4].bias,1)        
        nn.init.normal_(self.features[8].weight,mean=0,std=0.01)
        nn.init.constant_(self.features[8].bias,0)        
        nn.init.normal_(self.features[10].weight,mean=0,std=0.01)
        nn.init.constant_(self.features[10].bias,1)        
        nn.init.normal_(self.features[12].weight,mean=0,std=0.01)
        nn.init.constant_(self.features[12].bias,1)
        
        self.feature_outputs = [0]*len(self.features)
        self.switch_indices = dict()
        self.sizes = dict()
        
        # Deconv Part
        self.deconv_pool5 = nn.MaxUnpool2d(kernel_size=3, stride=2, padding=0)
        self.deconv_act5 = nn.ReLU()
        self.deconv_conv5 = nn.ConvTranspose2d(256,384,kernel_size=3,stride=1,padding=1,bias=False)
        self.deconv_act4 = nn.ReLU()
        self.deconv_conv4 = nn.ConvTranspose2d(384,384,kernel_size=3,stride=1,padding=1,bias=False)
        self.deconv_act3 = nn.ReLU()
        self.deconv_conv3 = nn.ConvTranspose2d(384,256,kernel_size=3,stride=1,padding=1,bias=False)
        self.deconv_pool2 = nn.MaxUnpool2d(kernel_size=3,stride=2,padding=1)
        self.deconv_act2 = nn.ReLU()
        self.deconv_conv2 = nn.ConvTranspose2d(256,96,kernel_size=5,stride=2,padding=0,bias=False)
        self.deconv_pool1 = nn.MaxUnpool2d(kernel_size=3,stride=2,padding=1)
        self.deconv_act1 = nn.ReLU()
        self.deconv_conv1 = nn.ConvTranspose2d(96,3,kernel_size=7,stride=2,padding=1,bias=False)
    
        self.layer6 = nn.Sequential(
            torch.nn.Dropout(p=0.5),                          #Implementing Dropout
            torch.nn.Linear(6*6*256,4096,bias=True),
            torch.nn.ReLU(),
        )
        self.layer7 = nn.Sequential(
            torch.nn.Dropout(p=0.5),                          #Implementing Dropout
            torch.nn.Linear(4096,4096,bias=True),
            torch.nn.ReLU()
        )
        self.layer8 = nn.Sequential(
            torch.nn.Dropout(p=0.5),                          #Implementing Dropout
            torch.nn.Linear(4096,2,bias=True),
        )
        torch.nn.init.normal_(self.layer6[1].weight,mean=0,std=0.01),
        torch.nn.init.constant_(self.layer6[1].bias,1)
        torch.nn.init.normal_(self.layer7[1].weight,mean=0,std=0.01),
        torch.nn.init.constant_(self.layer7[1].bias,1)
        torch.nn.init.normal_(self.layer8[1].weight,mean=0,std=0.01),
        torch.nn.init.constant_(self.layer8[1].bias,1)
        
    def forward(self, x):
        for i, layer in enumerate(self.features):
            if isinstance(layer,nn.MaxPool2d):
                x, indices = layer(x)
                self.feature_outputs[i] = x
                self.switch_indices[i] = indices
            else:
                x = layer(x)
                self.feature_outputs[i] = x
        x=x.view(x.size(0),-1)
        x=self.layer6(x)
        x=self.layer7(x)
        x=self.layer8(x)
        return x
    
    def forward_deconv(self,x,layer):
        if layer < 1 or layer > 5:
            raise Exception("ZFnet -> forward_deconv(): layer value should be between [1, 5]")
            
        x = self.deconv_pool5(x,self.switch_indices[12],output_size=self.feature_outputs[-2].shape[-2:])
        x = self.deconv_act5(x)
        x = self.deconv_conv5(x)
        
        if layer == 1:
            return x            

        x = self.deconv_act4(x)
        x = self.deconv_conv4(x)
        
        if layer == 2:
            return x
        
        x = self.deconv_act3(x)
        x = self.deconv_conv3(x)
        
        if layer == 3:
            return x
        
        x = self.deconv_pool2(x,self.switch_indices[5],output_size=self.feature_outputs[4].shape[-2:])
        x = self.deconv_act2(x)
        x = self.deconv_conv2(x)
        
        if layer == 4:
            return x

        x = self.deconv_pool1(x,self.switch_indices[2],output_size=self.feature_outputs[1].shape[-2:])
        x = self.deconv_act1(x)
        x = self.deconv_conv1(x)
        
        if layer == 5:
            return x
    
model = ZFNet().to(device)
summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 110, 110]          14,208
              ReLU-2         [-1, 96, 110, 110]               0
         MaxPool2d-3  [[-1, 96, 55, 55], [-1, 96, 55, 55]]               0
 LocalResponseNorm-4           [-1, 96, 55, 55]               0
            Conv2d-5          [-1, 256, 26, 26]         614,656
              ReLU-6          [-1, 256, 26, 26]               0
         MaxPool2d-7  [[-1, 256, 13, 13], [-1, 256, 13, 13]]               0
 LocalResponseNorm-8          [-1, 256, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]         885,120
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 384, 13, 13]       1,327,488
             ReLU-12          [-1, 384, 13, 13]               0
           Conv2d-13          [-1, 256, 13, 13]         884,992
             Re

In [2]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=lr,momentum=0.9,weight_decay=5e-4)
# Using SGD with L2 weight decay 0.0005, momentum 0.9

# total_batch = len(trainloader)
# print(total_batch)
# save_accuracy=torch.Tensor([]).to(device)

for epoch in range(40):
    avg_cost = 0
    for X,Y in trainloader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        correct_prediction = torch.argmax(hypothesis, 1) ==1
#         print(correct_prediction)
#         accuracy = correct_prediction.float().mean()
#         print(accuracy)
#         save_accuracy += accuracy
#         print(save_accuracy)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost/40
        break
#     acc = save_accuracy.mean()
#     print('Accuracy:', acc)
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.0389055982
[Epoch:    2] cost = 0.116402999


RuntimeError: CUDA out of memory. Tried to allocate 144.00 MiB (GPU 0; 1.95 GiB total capacity; 880.69 MiB already allocated; 109.81 MiB free; 1.02 GiB reserved in total by PyTorch)

In [ ]:
with torch.no_grad():
    a = 0
    for X,Y in testloader:
        X = X.to(device)
        Y = Y.to(device)
        prediction = model(X)
        correct_prediction = torch.argmax(prediction, 1) == Y
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())
        a += 1
        print(a)
        if a == 10:
            break